In [1]:
import dash
from dash import html, dcc, dash_table, callback_context
import plotly.graph_objects as go
import dash_trich_components as dtc
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px
import json
import pandas as pd
from pandas_ta import bbands
import pandas_datareader as web
import numpy as np
import datetime as dt
from scipy.stats import pearsonr
import yfinance as yf
from yahoo_fin import stock_info as si
import pandas as pd
import glob
import os
from pathlib import Path
import re

In [2]:
spxurl = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
spx_table = pd.read_html(spxurl)
spx_table = spx_table[0]
spx_table = spx_table.rename(columns={'Symbol':'Ticker'})
spx_table.head()
# tickers = spx_table[0]['Symbol'].tolist()

,Ticker,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [3]:
folder = '/home/creeg/projects/equities/equities/tickers'
path = folder+"/**/*.csv"
csv_paths = glob.glob(path, recursive=True)
dfs = [pd.read_csv(csv_path) for csv_path in csv_paths]
temp_df = pd.concat([pd.read_csv(fp).assign(Ticker=os.path.basename(fp).split('.')[0]) 
       for fp in csv_paths])

temp_df.head()


,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,Unnamed: 0,ticker,px_last,score,pe_ratio,peg_ratio,sma_150d,sma_200d,52week_low,52week_high
0,2022-06-01,4149.779785,4166.540039,4073.850098,4101.229980,4101.229980,4.531800e+09,AMD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-06-02,4095.409912,4177.509766,4074.370117,4176.819824,4176.819824,4.405790e+09,AMD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-06-03,4137.569824,4142.669922,4098.669922,4108.540039,4108.540039,3.711110e+09,AMD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-06-06,4134.720215,4168.779785,4109.180176,4121.430176,4121.430176,4.332700e+09,AMD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-06-07,4096.470215,4164.859863,4080.189941,4160.680176,4160.680176,4.248210e+09,AMD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
count = 0
# Iterate directory
for path in os.listdir(folder):
    # check if current path is a file
    if os.path.isfile(os.path.join(folder, path)):
        count += 1
print('File count:', count)

File count: 0


In [5]:
temp_df = temp_df.merge(spx_table, on='Ticker', how='left')
temp_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,Unnamed: 0,ticker,...,sma_200d,52week_low,52week_high,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,2022-06-01,4149.779785,4166.540039,4073.850098,4101.229980,4101.229980,4.531800e+09,AMD,NaN,NaN,...,NaN,NaN,NaN,AMD,Information Technology,Semiconductors,"Santa Clara, California",2017-03-20,2488.0,1969
1,2022-06-02,4095.409912,4177.509766,4074.370117,4176.819824,4176.819824,4.405790e+09,AMD,NaN,NaN,...,NaN,NaN,NaN,AMD,Information Technology,Semiconductors,"Santa Clara, California",2017-03-20,2488.0,1969
2,2022-06-03,4137.569824,4142.669922,4098.669922,4108.540039,4108.540039,3.711110e+09,AMD,NaN,NaN,...,NaN,NaN,NaN,AMD,Information Technology,Semiconductors,"Santa Clara, California",2017-03-20,2488.0,1969
3,2022-06-06,4134.720215,4168.779785,4109.180176,4121.430176,4121.430176,4.332700e+09,AMD,NaN,NaN,...,NaN,NaN,NaN,AMD,Information Technology,Semiconductors,"Santa Clara, California",2017-03-20,2488.0,1969
4,2022-06-07,4096.470215,4164.859863,4080.189941,4160.680176,4160.680176,4.248210e+09,AMD,NaN,NaN,...,NaN,NaN,NaN,AMD,Information Technology,Semiconductors,"Santa Clara, California",2017-03-20,2488.0,1969


In [6]:
cols = temp_df.columns.tolist()
cols

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Adj Close',
 'Volume',
 'Ticker',
 'Unnamed: 0',
 'ticker',
 'px_last',
 'score',
 'pe_ratio',
 'peg_ratio',
 'sma_150d',
 'sma_200d',
 '52week_low',
 '52week_high',
 'Security',
 'GICS Sector',
 'GICS Sub-Industry',
 'Headquarters Location',
 'Date added',
 'CIK',
 'Founded']

In [7]:
cols_ordered = [ 'Ticker'
        ,'Security'
        ,'GICS Sector'
        ,'GICS Sub-Industry'
        ,'Headquarters Location'
        ,'Date added'
        ,'CIK'
        ,'Founded'
        ,'Date'
        ,'Open'
        ,'High'
        ,'Low'
        ,'Close'
        ,'Adj Close'
        ,'Volume'
        ]

temp_df = temp_df[cols_ordered]
temp_df.tail()

,Ticker,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded,Date,Open,High,Low,Close,Adj Close,Volume
126248,CTVA,Corteva,Materials,Fertilizers & Agricultural Chemicals,"Indianapolis, Indiana",2019-06-03,1755672.0,2019,2023-05-24,4132.959961,4132.959961,4103.979980,4115.240234,4115.240234,3.739160e+09
126249,CTVA,Corteva,Materials,Fertilizers & Agricultural Chemicals,"Indianapolis, Indiana",2019-06-03,1755672.0,2019,2023-05-25,4155.709961,4165.740234,4129.729980,4151.279785,4151.279785,4.147760e+09
126250,CTVA,Corteva,Materials,Fertilizers & Agricultural Chemicals,"Indianapolis, Indiana",2019-06-03,1755672.0,2019,2023-05-26,4156.160156,4212.870117,4156.160156,4205.450195,4205.450195,3.715460e+09
126251,CTVA,Corteva,Materials,Fertilizers & Agricultural Chemicals,"Indianapolis, Indiana",2019-06-03,1755672.0,2019,2023-05-30,4226.709961,4231.100098,4192.180176,4205.520020,4205.520020,4.228510e+09
126252,CTVA,Corteva,Materials,Fertilizers & Agricultural Chemicals,"Indianapolis, Indiana",2019-06-03,1755672.0,2019,2023-05-31,4190.740234,4195.439941,4166.149902,4179.830078,4179.830078,5.980670e+09


In [8]:
temp_df.dtypes

Ticker                    object
Security                  object
GICS Sector               object
GICS Sub-Industry         object
Headquarters Location     object
Date added                object
CIK                      float64
Founded                   object
Date                      object
Open                     float64
High                     float64
Low                      float64
Close                    float64
Adj Close                float64
Volume                   float64
dtype: object

In [9]:
temp_df['Date'] = temp_df['Date'].apply(pd.to_datetime)

In [10]:
date_range = str(temp_df['Date'].dt.date.min()) + ' to ' + str(temp_df['Date'].dt.date.max())
date_range

'2022-06-01 to 2023-05-31'

In [11]:
temp_df.columns

Index(['Ticker', 'Security', 'GICS Sector', 'GICS Sub-Industry',
       'Headquarters Location', 'Date added', 'CIK', 'Founded', 'Date', 'Open',
       'High', 'Low', 'Close', 'Adj Close', 'Volume'],
      dtype='object')

In [12]:
temp_df['pct_change'] = temp_df.groupby(['Ticker'])['Adj Close'].pct_change(-1)

In [13]:
# tickers = si.tickers_sp500()
tickers = temp_df['Ticker'].unique()
tickers = tickers.tolist()
start = str(temp_df['Date'].dt.date.min())
end = str(temp_df['Date'].dt.date.max())

spx_df = yf.download(tickers=['^GSPC'], start=start, end=end)
spx_df['pct_change'] = spx_df['Adj Close'].pct_change()
spx_return = (spx_df['pct_change'] + 1).cumprod()[-1]

[*********************100%***********************]  1 of 1 completed


In [14]:
spx_return

1.025428966324531

In [15]:
spx_df.rename(columns={'pct_change':'spx_pct_chg'},inplace=True)

In [16]:
spx_df['Date'] = spx_df.index
spx_df_trunc = spx_df[['Date','spx_pct_chg']]
spx_df_trunc = spx_df_trunc.reset_index(drop=True)

In [17]:
temp_df = temp_df.merge(spx_df_trunc, on='Date', how='left')
temp_df.head()

,Ticker,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded,Date,Open,High,Low,Close,Adj Close,Volume,pct_change,spx_pct_chg
0,AMD,AMD,Information Technology,Semiconductors,"Santa Clara, California",2017-03-20,2488.0,1969,2022-06-01,4149.779785,4166.540039,4073.850098,4101.229980,4101.229980,4.531800e+09,-0.018097,NaN
1,AMD,AMD,Information Technology,Semiconductors,"Santa Clara, California",2017-03-20,2488.0,1969,2022-06-02,4095.409912,4177.509766,4074.370117,4176.819824,4176.819824,4.405790e+09,0.016619,0.018431
2,AMD,AMD,Information Technology,Semiconductors,"Santa Clara, California",2017-03-20,2488.0,1969,2022-06-03,4137.569824,4142.669922,4098.669922,4108.540039,4108.540039,3.711110e+09,-0.003128,-0.016347
3,AMD,AMD,Information Technology,Semiconductors,"Santa Clara, California",2017-03-20,2488.0,1969,2022-06-06,4134.720215,4168.779785,4109.180176,4121.430176,4121.430176,4.332700e+09,-0.009434,0.003137
4,AMD,AMD,Information Technology,Semiconductors,"Santa Clara, California",2017-03-20,2488.0,1969,2022-06-07,4096.470215,4164.859863,4080.189941,4160.680176,4160.680176,4.248210e+09,0.010912,0.009523


In [18]:
temp_df.columns

Index(['Ticker', 'Security', 'GICS Sector', 'GICS Sub-Industry',
       'Headquarters Location', 'Date added', 'CIK', 'Founded', 'Date', 'Open',
       'High', 'Low', 'Close', 'Adj Close', 'Volume', 'pct_change',
       'spx_pct_chg'],
      dtype='object')

In [19]:
temp_df['pct_chg_vs_spx'] = temp_df['pct_change'] - temp_df['spx_pct_chg']

In [20]:
spx_return

1.025428966324531

In [21]:
temp_df.isna().sum()

Ticker                      0
Security                 1004
GICS Sector              1004
GICS Sub-Industry        1004
Headquarters Location    1004
Date added               3514
CIK                      1004
Founded                  1004
Date                        0
Open                        0
High                        0
Low                         0
Close                       0
Adj Close                   0
Volume                      0
pct_change                503
spx_pct_chg              1006
pct_chg_vs_spx           1006
dtype: int64

In [22]:
nan_values = temp_df[temp_df['pct_change'].isna()]
print(nan_values)

       Ticker                   Security             GICS Sector  \
250       AMD                        AMD  Information Technology   
501       AON                        Aon              Financials   
752       ICE  Intercontinental Exchange              Financials   
1003      XOM                 ExxonMobil                  Energy   
1254      LYV  Live Nation Entertainment  Communication Services   
...       ...                        ...                     ...   
125248   AXON            Axon Enterprise             Industrials   
125499   STLD             Steel Dynamics               Materials   
125750    EIX       Edison International               Utilities   
126001   NVDA                     Nvidia  Information Technology   
126252   CTVA                    Corteva               Materials   

                           GICS Sub-Industry      Headquarters Location  \
250                           Semiconductors    Santa Clara, California   
501                        Insura

In [23]:
nan_values['Date'].nunique()

1

In [24]:
temp_df[['pct_change','spx_pct_chg','pct_chg_vs_spx']] = temp_df[['pct_change','spx_pct_chg','pct_chg_vs_spx']].fillna(0)

In [25]:
temp_df.isna().sum()

Ticker                      0
Security                 1004
GICS Sector              1004
GICS Sub-Industry        1004
Headquarters Location    1004
Date added               3514
CIK                      1004
Founded                  1004
Date                        0
Open                        0
High                        0
Low                         0
Close                       0
Adj Close                   0
Volume                      0
pct_change                  0
spx_pct_chg                 0
pct_chg_vs_spx              0
dtype: int64

In [26]:
nan_values = temp_df[temp_df['Security'].isna()]
nan_values['Ticker'].unique()

array(['FISV', 'BRK-B', 'BF-B', 'DISH'], dtype=object)

In [27]:
temp_df['GICS Sector'].unique()

array(['Information Technology', 'Financials', 'Energy',
       'Communication Services', 'Materials', 'Consumer Discretionary',
       'Industrials', 'Health Care', 'Consumer Staples', nan,
       'Real Estate', 'Utilities'], dtype=object)

In [28]:
temp_df['GICS Sub-Industry'].unique()

array(['Semiconductors', 'Insurance Brokers',
       'Financial Exchanges & Data', 'Integrated Oil & Gas',
       'Movies & Entertainment', 'Metal, Glass & Plastic Containers',
       'Advertising', 'Internet & Direct Marketing Retail',
       'Paper & Plastic Packaging Products & Materials',
       'Construction Machinery & Heavy Transportation Equipment',
       'Oil & Gas Storage & Transportation', 'Consumer Finance',
       'Pharmaceuticals', 'Consumer Staples Merchandise Retail', nan,
       'Internet Services & Infrastructure', 'Health Care Equipment',
       'Interactive Home Entertainment', 'Apparel Retail',
       'Specialty Chemicals', 'Systems Software',
       'Diversified Support Services', 'Health Care REITs',
       'Aerospace & Defense',
       'Industrial Machinery & Supplies & Components', 'Reinsurance',
       'Electric Utilities', 'Electronic Equipment & Instruments',
       'Publishing', 'Managed Health Care', 'Industrial Conglomerates',
       'Technology Hardware

In [29]:
gics = temp_df.groupby('GICS Sector')['GICS Sub-Industry'].apply(list).to_dict()

gics.keys()

dict_keys(['Communication Services', 'Consumer Discretionary', 'Consumer Staples', 'Energy', 'Financials', 'Health Care', 'Industrials', 'Information Technology', 'Materials', 'Real Estate', 'Utilities'])

In [30]:
list_industry = []

for key, value in gics.items():
    # if key in ['Financials', 'Information Technology', 'Communication Services','Consumer Staples']:
     if key in ['Financials']:
        # print(value).unique()
        list_industry.append(value)

set(list_industry[0])

{'Asset Management & Custody Banks',
 'Consumer Finance',
 'Diversified Banks',
 'Financial Exchanges & Data',
 'Insurance Brokers',
 'Investment Banking & Brokerage',
 'Life & Health Insurance',
 'Multi-line Insurance',
 'Property & Casualty Insurance',
 'Regional Banks',
 'Reinsurance',
 'Transaction & Payment Processing Services'}

In [31]:
temp_df[temp_df['Ticker'].str.contains('BRK')]

,Ticker,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded,Date,Open,High,Low,Close,Adj Close,Volume,pct_change,spx_pct_chg,pct_chg_vs_spx
52710,BRK-B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-01,4149.779785,4166.540039,4073.850098,4101.229980,4101.229980,4.531800e+09,-0.018097,0.000000,0.000000
52711,BRK-B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-02,4095.409912,4177.509766,4074.370117,4176.819824,4176.819824,4.405790e+09,0.016619,0.018431,-0.001812
52712,BRK-B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-03,4137.569824,4142.669922,4098.669922,4108.540039,4108.540039,3.711110e+09,-0.003128,-0.016347,0.013220
52713,BRK-B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-06,4134.720215,4168.779785,4109.180176,4121.430176,4121.430176,4.332700e+09,-0.009434,0.003137,-0.012571
52714,BRK-B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-07,4096.470215,4164.859863,4080.189941,4160.680176,4160.680176,4.248210e+09,0.010912,0.009523,0.001388
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52956,BRK-B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-24,4132.959961,4132.959961,4103.979980,4115.240234,4115.240234,3.739160e+09,-0.008682,-0.007319,-0.001363
52957,BRK-B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-25,4155.709961,4165.740234,4129.729980,4151.279785,4151.279785,4.147760e+09,-0.012881,0.008758,-0.021639
52958,BRK-B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-26,4156.160156,4212.870117,4156.160156,4205.450195,4205.450195,3.715460e+09,-0.000017,0.013049,-0.013066
52959,BRK-B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-30,4226.709961,4231.100098,4192.180176,4205.520020,4205.520020,4.228510e+09,0.006146,0.000017,0.006130


In [32]:
# 'FISV', 'BRK-B', 'BF-B', 'DISH'

nan_values_dict_sector = {'FISV': 'Information Technology'
                   ,'BRK-B': 'Financials'
                   ,'BF-B': 'Consumer Staples'
                   ,'DISH': 'Communication Services'
                   }

nan_values_dict_industry = {'FISV': 'Application Software'
                   ,'BRK-B': 'Multi-line Insurance'
                   ,'BF-B': 'Distillers & Vintners'
                   ,'DISH': 'Cable & Satellite'
                   }

nan_values_dict_security = {'FISV': 'Fiserv'
                   ,'BRK-B': 'Berkshire Hathaway Class B'
                   ,'BF-B': 'Brown-Forman Class B'
                   ,'DISH': 'DISH Network Corp'
                   }

In [33]:
temp_df['GICS Sector'] = temp_df['GICS Sector'].fillna(temp_df['Ticker'].map(nan_values_dict_sector))
temp_df['GICS Sub-Industry'] = temp_df['GICS Sub-Industry'].fillna(temp_df['Ticker'].map(nan_values_dict_industry))
temp_df['Security'] = temp_df['Security'].fillna(temp_df['Ticker'].map(nan_values_dict_security))

In [43]:
temp_df = temp_df.drop(['Headquarters Location','Date added','CIK','Founded'],axis=1)

In [44]:
temp_df.isna().sum()

Ticker               0
Security             0
GICS Sector          0
GICS Sub-Industry    0
Date                 0
Open                 0
High                 0
Low                  0
Close                0
Adj Close            0
Volume               0
pct_change           0
spx_pct_chg          0
pct_chg_vs_spx       0
dtype: int64